# Transformer-MIL Inference

This notebook reproduces the preprocessing and model architecture used for training, loads a saved checkpoint, and runs inference on unlabeled data to produce per-site probabilities.

Usage notes:
- Edit the parameters in the next code cell before running.
- The notebook will try to reuse a saved scaler if provided; otherwise it will fit a StandardScaler on the inference data (not ideal but a fallback).
- The model architecture is identical to training so that state dicts load cleanly.


In [17]:
# Imports
import gzip, json, os
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import joblib
from torch.cuda.amp import GradScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')


Using device: cuda


In [18]:
# Parameters (edit before running)
DATA_FILE = '../data/dataset0.json.gz'  # raw JSON.gz used at training time
MODEL_PATH = '../runs/run_20251029_105547/checkpoints/final_model_state_dict.pt'
SCALER_PATH = '../runs/run_20251029_105547/checkpoints/' 
OUTPUT_CSV = '../predictions/dataset0_predictions_transformers.csv'
BATCH_SIZE = 64
BAG_SIZE = 40
VERBOSE = True

In [19]:
# Load raw JSON.gz and build DataFrame of reads (no labels expected)
def load_unlabeled_data(data_file):
    rows = []
    with gzip.open(data_file, 'rt', encoding='utf-8') as f:
        total = sum(1 for _ in f)
    with gzip.open(data_file, 'rt', encoding='utf-8') as f:
        for line in tqdm(f, total=total, desc='Loading raw data'):
            data = json.loads(line)
            for transcript_id, positions in data.items():
                for transcript_position, sequences in positions.items():
                    for sequence, feature_list in sequences.items():
                        for features in feature_list:
                            rows.append({
                                'transcript_id': transcript_id,
                                'transcript_position': int(transcript_position),
                                'sequence': sequence,
                                'dwell_-1': features[0],
                                'std_-1': features[1],
                                'mean_-1': features[2],
                                'dwell_0': features[3],
                                'std_0': features[4],
                                'mean_0': features[5],
                                'dwell_+1': features[6],
                                'std_+1': features[7],
                                'mean_+1': features[8],
                            })
    df = pd.DataFrame(rows)
    # Add derived features same as training
    df['mean_0_minus_mean_-1'] = df['mean_0'] - df['mean_-1']
    df['mean_0_minus_mean_+1'] = df['mean_0'] - df['mean_+1']
    df['dwell_0_minus_dwell_-1'] = df['dwell_0'] - df['dwell_-1']
    df['dwell_0_minus_dwell_+1'] = df['dwell_0'] - df['dwell_+1']
    df['std_0_minus_avg_neighbor_std'] = df['std_0'] - ((df['std_-1'] + df['std_+1']) / 2.0)
    if VERBOSE:
        # Use string column names for grouping to avoid KeyError when variables of same names exist
        n_sites = df.groupby(['transcript_id', 'transcript_position']).ngroups if not df.empty else 0
        print(f'Loaded {len(df)} reads from {n_sites} sites')
    return df

# Run load
df_raw = load_unlabeled_data(DATA_FILE)
df_raw.head()

Loading raw data: 100%|██████████| 121838/121838 [01:00<00:00, 2007.41it/s]



Loaded 11027106 reads from 121838 sites


,transcript_id,transcript_position,sequence,dwell_-1,std_-1,mean_-1,dwell_0,std_0,mean_0,dwell_+1,std_+1,mean_+1,mean_0_minus_mean_-1,mean_0_minus_mean_+1,dwell_0_minus_dwell_-1,dwell_0_minus_dwell_+1,std_0_minus_avg_neighbor_std
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,-3.0,37.9,0.01471,0.00840,3.920
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,1.0,45.1,0.00213,-0.00186,0.255
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,15.0,44.4,0.00895,0.00862,8.975
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,5.0,49.6,0.00432,0.00332,2.090
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,9.0,46.8,-0.00398,-0.01034,-1.095


In [20]:
# Encoding helpers and numeric feature list
BASE2IDX = {'A':0, 'C':1, 'G':2, 'T':3, 'U':3}
PAD_IDX = 4

def seq_to_idx7(s: str):
    s = str(s).upper().replace('U', 'T')
    return np.array([BASE2IDX.get(ch, 0) for ch in s], dtype=np.int64)

# numeric columns used by model (must match training)
num_cols = [
    'dwell_-1','std_-1','mean_-1',
    'dwell_0','std_0','mean_0',
    'dwell_+1','std_+1','mean_+1',
    'mean_0_minus_mean_-1',
    'mean_0_minus_mean_+1',
    'dwell_0_minus_dwell_-1',
    'dwell_0_minus_dwell_+1',
    'std_0_minus_avg_neighbor_std'
]
seq_col = 'sequence'
site_key = ['transcript_id', 'transcript_position']


In [21]:
# Create bags for unlabeled data (labels set to 0 as placeholder)
def create_bags_unlabeled(df, site_key, num_cols, seq_col, min_reads=1, max_reads=50):
    bags = []
    grouped = df.groupby(site_key)
    for site, group in grouped:
        if len(group) < min_reads:
            continue
        features = group[num_cols].to_numpy(dtype=np.float32)
        sequences = group[seq_col].astype(str).tolist()
        seq_idx = np.vstack([seq_to_idx7(s) for s in sequences])
        n = len(features)
        if n > max_reads:
            idx = np.random.choice(n, max_reads, replace=False)
            features = features[idx]
            seq_idx = seq_idx[idx]
        bags.append({
            'site': site,
            'transcript_id': site[0],
            'transcript_position': int(site[1]),
            'features': features,
            'seq_idx': seq_idx,
            'n_reads': len(features),
            'label': 0,  # placeholder
            'gene_id': ''
        })
    print(f'Created {len(bags)} bags from {len(grouped)} sites')
    return bags

# Build bags from raw dataframe
bags = create_bags_unlabeled(df_raw, site_key, num_cols, seq_col, min_reads=1, max_reads=BAG_SIZE)


Created 121838 bags from 121838 sites


In [22]:
# Reuse RNA_MIL_Dataset but without oversampling (labels are placeholders)
class RNA_MIL_Dataset_Unlabeled(Dataset):
    def __init__(self, bags, bag_size=40, pad_idx=PAD_IDX):
        self.proc = []
        self.meta = []
        for bag in bags:
            num = bag['features']
            seq = bag['seq_idx']
            n = bag['n_reads']
            if n == 0:
                continue
            if n < bag_size:
                pad_num = np.zeros((bag_size - n, num.shape[1]), dtype=np.float32)
                pad_seq = np.full((bag_size - n, seq.shape[1]), pad_idx, dtype=np.int64)
                num_fixed = np.vstack([num, pad_num])
                seq_fixed = np.vstack([seq, pad_seq])
                mask = np.zeros(bag_size, dtype=np.float32)
                mask[:n] = 1.0
            else:
                if n > bag_size:
                    idx = np.arange(n)[:bag_size]
                    num_fixed = num[idx].astype(np.float32)
                    seq_fixed = seq[idx].astype(np.int64)
                    mask = np.ones(bag_size, dtype=np.float32)
                else:
                    num_fixed = num.astype(np.float32)
                    seq_fixed = seq.astype(np.int64)
                    mask = np.ones(bag_size, dtype=np.float32)
            self.proc.append({'num': num_fixed, 'seq': seq_fixed, 'mask': mask})
            self.meta.append({'transcript_id': bag['transcript_id'], 'transcript_position': bag['transcript_position'], 'n_reads': bag['n_reads']})
    def __len__(self):
        return len(self.proc)
    def __getitem__(self, idx):
        b = self.proc[idx]
        x_num = torch.from_numpy(b['num'])
        x_seq = torch.from_numpy(b['seq'])
        mask  = torch.from_numpy(b['mask'])
        return x_num, x_seq, mask
    def get_metadata(self, idx):
        return self.meta[idx]

# Create dataset and dataloader
dataset = RNA_MIL_Dataset_Unlabeled(bags, bag_size=BAG_SIZE, pad_idx=PAD_IDX)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
print(f'Dataset size: {len(dataset)} bags, DataLoader batches: {len(dataloader)}')


Dataset size: 121838 bags, DataLoader batches: 1904


In [23]:
# Model classes (copied from training notebook, unchanged)
class SeqEmbCNN(nn.Module):
    def __init__(self, vocab=5, d_emb=8, kernel_sizes=(2,3,4,5), n_filters=32, d_out=64):
        super().__init__()
        self.emb = nn.Embedding(vocab, d_emb, padding_idx=PAD_IDX)
        self.convs = nn.ModuleList([
            nn.Conv1d(d_emb, n_filters, ks, padding=0) for ks in kernel_sizes
        ])
        self.attentions = nn.ModuleList([
            nn.Linear(n_filters, 1) for _ in kernel_sizes
        ])
        self.proj = nn.Linear(n_filters * len(kernel_sizes), d_out)
        self.norm = nn.LayerNorm(d_out)
    def forward(self, x_idx):
        X = self.emb(x_idx).transpose(1,2)
        feats = []
        for conv, att in zip(self.convs, self.attentions):
            h = torch.nn.functional.gelu(conv(X))
            h_t = h.permute(0,2,1)
            scores = att(h_t).squeeze(-1)
            weights = torch.nn.functional.softmax(scores, dim=1).unsqueeze(-1)
            pooled = (h_t * weights).sum(dim=1)
            feats.append(pooled)
        z = torch.cat(feats, dim=1)
        z = self.proj(z)
        return self.norm(torch.nn.functional.gelu(z))

class GatedAttentionPooling(nn.Module):
    def __init__(self, d_model, n_heads=4, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.attention_heads = nn.ModuleList([
            nn.Sequential(nn.Linear(d_model, d_model), nn.Tanh(), nn.Dropout(dropout), nn.Linear(d_model,1)) for _ in range(n_heads)
        ])
        self.gate = nn.Sequential(nn.Linear(d_model, d_model//2), nn.ReLU(), nn.Dropout(dropout), nn.Linear(d_model//2,1), nn.Sigmoid())
        self.fusion = nn.Sequential(nn.Linear(d_model * n_heads, d_model), nn.LayerNorm(d_model), nn.GELU())
    def forward(self, h, mask):
        gates = self.gate(h).squeeze(-1) * mask
        pooled = []
        all_weights = []
        for attn in self.attention_heads:
            scores = attn(h).squeeze(-1)
            gated_scores = scores * gates
            gated_scores = gated_scores.masked_fill(mask == 0, float('-inf'))
            weights = torch.nn.functional.softmax(gated_scores, dim=1).unsqueeze(-1)
            pooled.append((h * weights).sum(dim=1))
            all_weights.append(weights.squeeze(-1))
        bag_repr = self.fusion(torch.cat(pooled, dim=-1))
        avg_weights = torch.stack(all_weights, dim=0).mean(dim=0)
        return bag_repr, avg_weights, gates

class TransformerMIL(nn.Module):
    def __init__(self, num_features=9, d_model=128, n_heads=4, n_layers=4, d_ff=1024, dropout=0.1, attn_pool_heads=4, instance_dropout=0.15):
        super().__init__()
        self.instance_dropout = instance_dropout
        self.seq_encoder = SeqEmbCNN(vocab=5, d_emb=8, kernel_sizes=(2,3,4,5), n_filters=32, d_out=64)
        self.num_proj = nn.Sequential(nn.Linear(num_features, 64), nn.LayerNorm(64), nn.GELU(), nn.Dropout(dropout))
        self.feature_fusion = nn.Sequential(nn.Linear(128, d_model), nn.LayerNorm(d_model), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_model, d_model), nn.LayerNorm(d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads, dim_feedforward=d_ff, dropout=dropout, activation='gelu', batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.attention_pool = GatedAttentionPooling(d_model, n_heads=attn_pool_heads, dropout=dropout)
        self.classifier = nn.Sequential(nn.Linear(d_model, d_model//2), nn.LayerNorm(d_model//2), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_model//2, d_model//4), nn.LayerNorm(d_model//4), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_model//4, 1))
        self.instance_classifier = nn.Sequential(nn.Linear(d_model, d_model//2), nn.LayerNorm(d_model//2), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_model//2, d_model//4), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_model//4, 1))
    def encode_sequences(self, x_seq):
        B, K, L = x_seq.shape
        seq_flat = x_seq.reshape(B * K, L)
        z_seq = self.seq_encoder(seq_flat)
        return z_seq.view(B, K, -1)
    def apply_instance_dropout(self, mask):
        if not self.training or self.instance_dropout == 0:
            return mask
        B, K = mask.shape
        dropout_mask = torch.rand(B, K, device=mask.device) > self.instance_dropout
        n_keep = (dropout_mask * mask).sum(dim=1, keepdim=True)
        too_few = n_keep < 5
        dropout_mask = torch.where(too_few, torch.ones_like(dropout_mask), dropout_mask)
        return mask * dropout_mask.float()
    def forward(self, x_num, mask, x_seq=None):
        B, K, _ = x_num.shape
        effective_mask = self.apply_instance_dropout(mask)
        num_features = self.num_proj(x_num)
        if x_seq is not None:
            seq_features = self.encode_sequences(x_seq)
            combined = torch.cat([num_features, seq_features], dim=-1)
        else:
            combined = num_features
        h = self.feature_fusion(combined)
        src_key_padding_mask = (effective_mask == 0)
        h = self.transformer(h, src_key_padding_mask=src_key_padding_mask)
        instance_logits = self.instance_classifier(h).squeeze(-1)
        instance_logits = instance_logits.masked_fill(effective_mask == 0, float('-inf'))
        bag_repr, attention_weights, gates = self.attention_pool(h, effective_mask)
        bag_logits = self.classifier(bag_repr).squeeze(-1)
        return bag_logits, attention_weights, instance_logits, gates


In [24]:
# Instantiate model and load weights
model = TransformerMIL(num_features=len(num_cols))
device = torch.device(DEVICE)
model = model.to(device)

# Try loading different checkpoint formats
if MODEL_PATH is not None and Path(MODEL_PATH).exists():
    ckpt = torch.load(MODEL_PATH, map_location=device)
    # If it's a plain state_dict (mapping of tensors), load directly
    if isinstance(ckpt, dict) and any(k.startswith('model') or k.endswith('state_dict') for k in ckpt.keys()):
        # support different key names
        if 'model_state' in ckpt:
            state = ckpt['model_state']
        elif 'model_state_dict' in ckpt:
            state = ckpt['model_state_dict']
        elif 'model' in ckpt and isinstance(ckpt['model'], dict):
            state = ckpt['model']
        else:
            # assume ckpt itself is state_dict-like
            state = ckpt
    else:
        state = ckpt
    model.load_state_dict(state)
    print(f'Loaded model weights from {MODEL_PATH}')
else:
    print(f'MODEL_PATH does not exist: {MODEL_PATH}. Please set MODEL_PATH to a valid checkpoint.')


Loaded model weights from ../runs/run_20251029_105547/checkpoints/final_model_state_dict.pt


In [25]:
# Scaler loading (explicit scaler_path and amp_path; minimal/no error-handling)
# Define explicit paths (prefer SCALER_PATH if provided, else candidate next to MODEL_PATH)
scaler_path = 'scaler.joblib'
if SCALER_PATH:
    p = Path(SCALER_PATH)
    scaler_path = p if p.is_file() else (p / 'scaler.joblib')
elif MODEL_PATH:
    scaler_path = Path(MODEL_PATH).parent / 'scaler.joblib'

amp_path = 'amp_grad_scaler.pt'
if SCALER_PATH:
    p = Path(SCALER_PATH)
    amp_path = (p / 'amp_grad_scaler.pt') if p.is_dir() else (p if p.name == 'amp_grad_scaler.pt' else None)
elif MODEL_PATH:
    amp_path = Path(MODEL_PATH).parent / 'amp_grad_scaler.pt'

# Load feature scaler if scaler_path exists
feature_scaler = None
saved_num_cols = None
if scaler_path is not None and Path(scaler_path).exists():
    payload = joblib.load(scaler_path)
    if isinstance(payload, dict) and 'scaler' in payload:
        feature_scaler = payload['scaler']
        saved_num_cols = payload.get('num_cols', None)
    else:
        feature_scaler = payload
    print(f'Loaded feature scaler from {scaler_path}')
else:
    print('No scaler_path found; feature_scaler remains None')

# Load AMP GradScaler directly if amp_path exists (minimal loading)
amp_scaler = None
if amp_path is not None and Path(amp_path).exists():
    amp_state = torch.load(amp_path, map_location='cpu')
    amp_scaler = GradScaler()
    amp_scaler.load_state_dict(amp_state)
    print(f'Loaded AMP GradScaler from {amp_path}')
else:
    print('No amp_path found; amp_scaler remains None')

# Verify column consistency when possible
if feature_scaler is not None and saved_num_cols is not None:
    if list(saved_num_cols) != list(num_cols):
        print('WARNING: numeric column list in saved scaler differs from current num_cols.')
        print(f'Saved: {saved_num_cols}')
        print(f'Current: {num_cols}')
    else:
        print('Saved scaler column order matches current num_cols')

# Apply scaling if we have a feature scaler
if feature_scaler is not None:
    df_raw[num_cols] = feature_scaler.transform(df_raw[num_cols])
    print('Applied feature scaler to df_raw')

# Rebuild bags and dataset now that df_raw is scaled
bags = create_bags_unlabeled(df_raw, site_key, num_cols, seq_col, min_reads=1, max_reads=BAG_SIZE)
dataset = RNA_MIL_Dataset_Unlabeled(bags, bag_size=BAG_SIZE, pad_idx=PAD_IDX)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
print(f'Dataset size: {len(dataset)} bags, DataLoader batches: {len(dataloader)}')


Loaded feature scaler from ..\runs\run_20251029_105547\checkpoints\scaler.joblib
Loaded AMP GradScaler from ..\runs\run_20251029_105547\checkpoints\amp_grad_scaler.pt


C:\Users\inbam\AppData\Local\Temp\ipykernel_14484\261539347.py:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  amp_scaler = GradScaler()


Applied feature scaler to df_raw
Created 121838 bags from 121838 sites
Created 121838 bags from 121838 sites
Dataset size: 121838 bags, DataLoader batches: 1904
Dataset size: 121838 bags, DataLoader batches: 1904


In [26]:
# Diagnostic 1 — checkpoint & model state mapping
import torch, pprint
from pathlib import Path

print('MODEL_PATH ->', MODEL_PATH)
if MODEL_PATH is None or not Path(MODEL_PATH).exists():
    print('MODEL_PATH is missing or does not exist. Aborting checkpoint diagnostic.')
else:
    ckpt = torch.load(MODEL_PATH, map_location='cpu')
    print('Checkpoint type:', type(ckpt))
    if isinstance(ckpt, dict):
        print('Top-level keys:', list(ckpt.keys()))
    # locate candidate state dict
    def find_state_dict(obj):
        if isinstance(obj, dict) and obj and all(isinstance(v, torch.Tensor) for v in list(obj.values())[:5]):
            return obj
        for k in ['model_state_dict','model_state','state_dict','model']:
            if isinstance(obj, dict) and k in obj and isinstance(obj[k], dict):
                return obj[k]
        return None
    state = find_state_dict(ckpt) or (ckpt if isinstance(ckpt, dict) and any(isinstance(v, torch.Tensor) for v in ckpt.values()) else None)
    if state is None:
        print('Could not find a state-dict-like object in the checkpoint.')
    else:
        state_keys = list(state.keys())
        print(f'Found state dict with {len(state_keys)} keys; sample keys:')
        pprint.pprint(state_keys[:20])
        # instantiate a fresh model with current num_cols
        model_tmp = TransformerMIL(num_features=len(num_cols))
        model_keys = list(model_tmp.state_dict().keys())
        print(f'Current model expects {len(model_keys)} keys; sample:')
        pprint.pprint(model_keys[:20])
        # check missing/unexpected when loading with strict=False
        load_res = model_tmp.load_state_dict(state, strict=False)
        print('load_state_dict(strict=False) result:')
        print('  missing_keys (keys required by model but absent in state):', len(load_res.missing_keys))
        if load_res.missing_keys:
            print('    sample missing:', load_res.missing_keys[:10])
        print('  unexpected_keys (keys present in state but not used by model):', len(load_res.unexpected_keys))
        if load_res.unexpected_keys:
            print('    sample unexpected:', load_res.unexpected_keys[:10])
        model_param_count = sum(p.numel() for p in model_tmp.parameters())
        print(f'Current model total params: {model_param_count:,}')

MODEL_PATH -> ../runs/run_20251029_105547/checkpoints/final_model_state_dict.pt
Checkpoint type: <class 'collections.OrderedDict'>
Top-level keys: ['seq_encoder.emb.weight', 'seq_encoder.convs.0.weight', 'seq_encoder.convs.0.bias', 'seq_encoder.convs.1.weight', 'seq_encoder.convs.1.bias', 'seq_encoder.convs.2.weight', 'seq_encoder.convs.2.bias', 'seq_encoder.convs.3.weight', 'seq_encoder.convs.3.bias', 'seq_encoder.attentions.0.weight', 'seq_encoder.attentions.0.bias', 'seq_encoder.attentions.1.weight', 'seq_encoder.attentions.1.bias', 'seq_encoder.attentions.2.weight', 'seq_encoder.attentions.2.bias', 'seq_encoder.attentions.3.weight', 'seq_encoder.attentions.3.bias', 'seq_encoder.proj.weight', 'seq_encoder.proj.bias', 'seq_encoder.norm.weight', 'seq_encoder.norm.bias', 'num_proj.0.weight', 'num_proj.0.bias', 'num_proj.1.weight', 'num_proj.1.bias', 'feature_fusion.0.weight', 'feature_fusion.0.bias', 'feature_fusion.1.weight', 'feature_fusion.1.bias', 'feature_fusion.4.weight', 'featur

In [27]:
# Diagnostic 2 — scaler and feature statistics
import joblib, numpy as np
from pathlib import Path

# Resolve scaler path same logic as notebook
scaler_path = Path('scaler.joblib')
if SCALER_PATH:
    p = Path(SCALER_PATH)
    scaler_path = p if p.is_file() else (p / 'scaler.joblib')
elif MODEL_PATH:
    scaler_path = Path(MODEL_PATH).parent / 'scaler.joblib'

print('Resolved scaler_path ->', scaler_path)
if not scaler_path.exists():
    raise FileNotFoundError(f'Scaler file not found at path: {scaler_path}.')
payload = joblib.load(scaler_path)
print('Loaded scaler payload type:', type(payload))
if isinstance(payload, dict):
    print('Payload keys:', list(payload.keys()))
    feature_scaler = payload.get('scaler', payload)
    saved_cols = payload.get('num_cols', None)
else:
    feature_scaler = payload
    saved_cols = None
print('saved_num_cols ->', saved_cols)
print('current num_cols ->', num_cols)
if hasattr(feature_scaler, 'mean_'):
    print('scaler mean_ (first 10):', np.round(feature_scaler.mean_[:10], 6))
if hasattr(feature_scaler, 'scale_'):
    print('scaler scale_ (first 10):', np.round(feature_scaler.scale_[:10], 6))
print('Sampling raw data stats (first batch of reads)')
df_sample = load_unlabeled_data(DATA_FILE)
if len(df_sample) == 0:
    raise RuntimeError('No reads found in DATA_FILE')
import pandas as pd
print('Raw feature describe:')
display(df_sample[num_cols].describe())
transformed = feature_scaler.transform(df_sample[num_cols]) if feature_scaler is not None else None
if transformed is not None:
    print('Transformed feature describe (first 10 cols):')
    print(pd.DataFrame(transformed, columns=num_cols).describe().transpose().head(10))
    print('Any NaNs after transform? ->', np.isnan(transformed).any())

Resolved scaler_path -> ..\runs\run_20251029_105547\checkpoints\scaler.joblib
Loaded scaler payload type: <class 'sklearn.preprocessing._data.StandardScaler'>
saved_num_cols -> None
current num_cols -> ['dwell_-1', 'std_-1', 'mean_-1', 'dwell_0', 'std_0', 'mean_0', 'dwell_+1', 'std_+1', 'mean_+1', 'mean_0_minus_mean_-1', 'mean_0_minus_mean_+1', 'dwell_0_minus_dwell_-1', 'dwell_0_minus_dwell_+1', 'std_0_minus_avg_neighbor_std']
scaler mean_ (first 10): [8.11200000e-03 4.34151800e+00 1.10902820e+02 8.14000000e-03
 5.16849800e+00 1.11011468e+02 7.07700000e-03 2.97629800e+00
 8.62754340e+01 1.08648000e-01]
scaler scale_ (first 10): [5.4180000e-03 2.5795620e+00 1.1918103e+01 5.3000000e-03 3.0072430e+00
 1.3004381e+01 4.3860000e-03 1.6960540e+00 5.5726260e+00 1.5378791e+01]
Sampling raw data stats (first batch of reads)


Loading raw data: 100%|██████████| 121838/121838 [00:48<00:00, 2497.72it/s]



Loaded 11027106 reads from 121838 sites
Raw feature describe:
Raw feature describe:


,dwell_-1,std_-1,mean_-1,dwell_0,std_0,mean_0,dwell_+1,std_+1,mean_+1,mean_0_minus_mean_-1,mean_0_minus_mean_+1,dwell_0_minus_dwell_-1,dwell_0_minus_dwell_+1,std_0_minus_avg_neighbor_std
count,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07,1.102711e+07
mean,8.106457e-03,4.347531e+00,1.109025e+02,8.149364e-03,5.182957e+00,1.110816e+02,7.077341e-03,2.981141e+00,8.625382e+01,1.791089e-01,2.482776e+01,4.290677e-05,1.072024e-03,1.518621e+00
std,5.411981e-03,2.577925e+00,1.192639e+01,5.301427e-03,3.008630e+00,1.302800e+01,4.385762e-03,1.699636e+00,5.556484e+00,1.541130e+01,1.537541e+01,7.571703e-03,6.759301e-03,3.202656e+00
min,1.660000e-03,9.400000e-02,7.320000e+01,1.660000e-03,4.400000e-02,7.540000e+01,1.660000e-03,1.360000e-01,6.100000e+01,-6.710000e+01,-4.470000e+01,-1.152800e-01,-9.736000e-02,-9.719000e+01
25%,4.320000e-03,2.500000e+00,1.040000e+02,4.490000e-03,2.930000e+00,9.870000e+01,4.090000e-03,1.930000e+00,8.200000e+01,-7.000000e+00,9.800000e+00,-3.760000e-03,-2.430000e-03,-4.600000e-01
50%,6.640000e-03,3.550000e+00,1.110000e+02,6.690000e-03,4.230000e+00,1.120000e+02,5.980000e-03,2.510000e+00,8.640000e+01,2.000000e+00,2.770000e+01,6.000000e-05,6.700000e-04,9.450000e-01
75%,1.020000e-02,5.570000e+00,1.200000e+02,1.030000e-02,6.890000e+00,1.230000e+02,8.630000e-03,3.470000e+00,9.040000e+01,1.000000e+01,3.890000e+01,3.940000e-03,4.310000e-03,3.135000e+00
max,1.200000e-01,2.060000e+02,1.530000e+02,1.380000e-01,2.060000e+02,1.560000e+02,1.030000e-01,1.840000e+02,1.430000e+02,5.570000e+01,7.660000e+01,1.346800e-01,1.297000e-01,2.006400e+02


Transformed feature describe (first 10 cols):
                           count      mean       std       min       25%  \
dwell_-1              11027106.0 -0.001053  0.998892 -1.190879 -0.699922   
std_-1                11027106.0  0.002331  0.999365 -1.646604 -0.713888   
mean_-1               11027106.0 -0.000029  1.000695 -3.163492 -0.579188   
dwell_0               11027106.0  0.001728  1.000354 -1.222784 -0.688776   
std_0                 11027106.0  0.004808  1.000461 -1.704052 -0.744369   
mean_0                11027106.0  0.005391  1.001816 -2.738421 -0.946717   
dwell_+1              11027106.0  0.000146  0.999885 -1.234922 -0.680920   
std_+1                11027106.0  0.002855  1.002112 -1.674651 -0.616902   
mean_+1               11027106.0 -0.003879  0.997103 -4.535641 -0.767221   
mean_0_minus_mean_-1  11027106.0  0.004582  1.002114 -4.370217 -0.462237   

                           50%       75%         max  
dwell_-1             -0.271718  0.385353   20.651192  
std_-1 

In [28]:
# Diagnostic 3 — small forward pass and logits/probability stats
import torch
model.eval()
print('Taking a single batch from dataloader (no shuffling)')
batch = next(iter(dataloader))
x_num, x_seq, mask = batch
x_num = x_num.to(device)
x_seq = x_seq.to(device)
mask  = mask.to(device)
with torch.no_grad():
    bag_logits, attn, inst_logits, gates = model(x_num=x_num, x_seq=x_seq, mask=mask)
logits = bag_logits.cpu().numpy()
probs = torch.sigmoid(torch.tensor(logits)).numpy()
import numpy as _np
print('logits -> mean: {:.6f}, std: {:.6f}, min: {:.6f}, max: {:.6f}'.format(_np.mean(logits), _np.std(logits), _np.min(logits), _np.max(logits)))
print('probs -> mean: {:.6f}, std: {:.6f}, min: {:.6f}, max: {:.6f}'.format(_np.mean(probs), _np.std(probs), _np.min(probs), _np.max(probs)))
print('Sample logits (first 10):', logits[:10])
print('Sample probs (first 10):', probs[:10])
for i in range(min(10, len(probs))):
    meta = dataset.get_metadata(i)
    print(f'idx={i} meta={meta} prob={float(probs[i]):.6f} logits={float(logits[i]):.6f}')
if _np.allclose(logits, logits[0], atol=1e-6):
    print('All logits nearly identical — possible causes: constant inputs, collapsed model, or scale mismatch.')
param_means = [p.detach().cpu().numpy().mean() for p in model.parameters() if p.numel() > 0]
print('Model parameter means (sample 10):', param_means[:10])

Taking a single batch from dataloader (no shuffling)
logits -> mean: -1.327085, std: 1.707713, min: -3.522403, max: 3.649587
probs -> mean: 0.273978, std: 0.274931, min: 0.028681, max: 0.974657
Sample logits (first 10): [-3.363253   -1.7439674  -3.4965966   0.368535   -2.7977657  -0.25326368
 -2.7917993  -2.1160352   1.2464864  -1.9528238 ]
Sample probs (first 10): [0.03346385 0.1488097  0.02940922 0.5911049  0.05744503 0.4370204
 0.05776894 0.10754802 0.776691   0.12424579]
idx=0 meta={'transcript_id': 'ENST00000000233', 'transcript_position': 244, 'n_reads': 40} prob=0.033464 logits=-3.363253
idx=1 meta={'transcript_id': 'ENST00000000233', 'transcript_position': 261, 'n_reads': 40} prob=0.148810 logits=-1.743967
idx=2 meta={'transcript_id': 'ENST00000000233', 'transcript_position': 316, 'n_reads': 40} prob=0.029409 logits=-3.496597
idx=3 meta={'transcript_id': 'ENST00000000233', 'transcript_position': 332, 'n_reads': 40} prob=0.591105 logits=0.368535
idx=4 meta={'transcript_id': 'ENS

In [29]:
# Run inference and save per-site probabilities
model.eval()
results = []
with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(dataloader, desc='Running inference')):
        x_num, x_seq, mask = batch
        x_num = x_num.to(device)
        x_seq = x_seq.to(device)
        mask  = mask.to(device)
        bag_logits, attn, inst_logits, gates = model(x_num=x_num, x_seq=x_seq, mask=mask)
        probs = torch.sigmoid(bag_logits).cpu().numpy()
        batch_start = batch_idx * BATCH_SIZE
        for i in range(len(probs)):
            meta = dataset.get_metadata(batch_start + i)
            results.append({
                'transcript_id': meta['transcript_id'],
                'transcript_position': meta['transcript_position'],
                'score': float(probs[i])
            })

# Build DataFrame and save
pred_df = pd.DataFrame(results)
out_path = Path(OUTPUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)
pred_df.to_csv(out_path, index=False)
print(f'Saved predictions to {out_path}')
pred_df.head()


Running inference: 100%|██████████| 1904/1904 [00:28<00:00, 67.74it/s]



Saved predictions to ..\predictions\dataset0_predictions_transformers.csv


,transcript_id,transcript_position,score
0,ENST00000000233,244,0.033464
1,ENST00000000233,261,0.148810
2,ENST00000000233,316,0.029409
3,ENST00000000233,332,0.591105
4,ENST00000000233,368,0.057445
